In [2]:
import numpy as np
from scipy.spatial import distance
import pandas as pd
csv = pd.read_csv('baseball.csv',na_values='?')
#=================PRIMER PUTNO=====================

def binarizeCategoricalAttributeVector(v):
    return pd.get_dummies(v).values

def getCategoricalAttributes(df):
    return df.select_dtypes(exclude=np.number)
    
def readFrameAsMatrix(data):
    Vectors=[]
    M=data.values
    for i in data:
        Vectors.append(binarizeCategoricalAttributeVector(csv[[i]]))
    for r in range(0,M.shape[0]):
        for c in range(0,M.shape[1]):
            M[r,c]=Vectors[c][r]
    
    return M


In [5]:
#=================SEGUNDO PUTNO=====================(SE DEMORA UN POCO)
csv = pd.read_csv('baseball.csv')
M=readFrameAsMatrix(csv)
n=np.where(csv =='?')
a=csv.values
for i in range (0,4,2):# pongo 4 para comprobar los primeros 4 ? ---- n[0].shape[0],2): para todos los ?
    row=n[0][i]
    col=n[1][i]
    rowNear=-1
    colNear=-1
    min =np.inf
    for r in range(0, M.shape[0]):
        temp=0
        if r!=row and csv.values[r][0]==csv.values[row][0]:
            for c in range(0,M.shape[1]-4):
                if csv.values[r][c]!='?':
                    temp+=distance.euclidean(M[r][c],M[row][c])
                else:
                    temp+=np.inf
            if temp<min:
                rowNear=r
                colNear=col
                min=temp
        a[row][col]=csv.values[rowNear][col]
    a[row][col+1]=csv.values[rowNear][col+1]
#comparacion de data antigua vs nueva
print(csv.values[0])
print(a[0])



c:\users\davidherrera\pycharmprojects\bi1\venv\lib\site-packages\pandas\core\ops\__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


['ARI' 'NL' 2012 734 688 81 0.328 0.418 0.259 0 '?' '?' 162 '0.317'
 '0.415']
['ARI' 'NL' 2012 734 688 81 0.328 0.418 0.259 0 '5' '4' 162 '0.317'
 '0.415']


In [31]:
#=================QUINTO PUTNO=====================

import numpy as np
from matplotlib import pyplot as plt
import pandas as pd 
from scipy.stats import chi2
data =np.random.randint(10, size=( 5, 2))
# data = [['Alexander', 10],
#         ['Alexander',36],
#         ['male', 10],
#         ['male', 10]]
csv = pd.read_csv('baseball.csv')
def  getContingencyTable(arr):
    df1=pd.DataFrame(arr)
    return  pd.crosstab(df1.iloc[:,0],
                                df1.iloc[:,1],
                                margins = True,
                                rownames=['Cat. 1 --'],
                                colnames=['Cat. 2 --'])    
     

def computeExpectedOccurrences(ct):
    newTab=np.random.uniform(5,10,[ct.shape[0]-1,ct.shape[1]-1])
    for i in range(0,ct.shape[0]-1):
        for j in range(0,ct.shape[1]-1):
            newTab[i][j]=(ct.iloc[i]['All']*ct.loc['All'].iloc[j])/ct['All']['All']
    df2=pd.DataFrame(newTab)
    return df2

def computeChiSquare(arr2):
    contingency=getContingencyTable(arr2)
    occurrences=computeExpectedOccurrences(contingency)
    val=0
    for i in range(0,contingency.shape[0]-1):
        for j in range(0,contingency.shape[1]-1):
            val+=(contingency.iloc[i,j]-occurrences.iloc[i,j])**2/occurrences.iloc[i,j]
    return val

def checkIndependence(df, c1, c2, pValue):
    contingency = pd.crosstab(df[df.columns[c1]],df[df.columns[c2]],margins = True,rownames=['Cat. 1 --'],colnames=['Cat. 2 --'])
    occurrences=computeExpectedOccurrences(contingency)
    val=0
    for i in range(0,contingency.shape[0]-1):
        for j in range(0,contingency.shape[1]-1):
            val+=(contingency.iloc[i,j]-occurrences.iloc[i,j])**2/occurrences.iloc[i,j]
    grados = (df.shape[1]-1)*(df.shape[0]-1)
    critical = chi2.ppf(1-pValue, grados)
    return  val<critical,grados,val,critical

print('Matriz inicial')
print(data)

print('Tabla de contingencia')
tab=getContingencyTable(data)
print(tab)

print("Ocurrencias esperadas")
print(computeExpectedOccurrences(tab))

print('X^2')
print(computeChiSquare(data))

print(checkIndependence(csv,0,2,0.05)[0])

Matriz inicial
[[4 6]
 [6 2]
 [9 1]
 [6 3]
 [1 1]]
Tabla de contingencia
Cat. 2 --  1  2  3  6  All
Cat. 1 --                 
1          1  0  0  0    1
4          0  0  0  1    1
6          0  1  1  0    2
9          1  0  0  0    1
All        2  1  1  1    5
Ocurrencias esperadas
     0    1    2    3
0  0.4  0.2  0.2  0.2
1  0.4  0.2  0.2  0.2
2  0.8  0.4  0.4  0.4
3  0.4  0.2  0.2  0.2
X^2
10.0
True


In [ ]:
#============= Comprobar baseball ====================

labels=[]
for i in csv:
    labels.append(i)
for i in range(0,len(labels)-9):
    for j in range(i,len(labels)-9):
        bol,grados,chi,critical=checkIndependence(csv,i,j,0.05)
        print(('entre',labels[i],' y ',labels[j])if (bol)else'.')
        if bol==False:
            mu = 1000
            x = np.linspace(-chi, 5*critical/4+chi, 20)
            fig, ax = plt.subplots(figsize=(5, 3.75))
            fig.subplots_adjust(bottom=0.12)
            dist = chi2(grados, mu)
            plt.plot(x, dist.pdf(x), ls='-', c='black',label=r'$k=%i$ entre %s y %s' % (grados,labels[i],labels[j]))
            plt.plot(chi,0,'bo')
            plt.plot(critical,0,'go')
            plt.xlim(-chi,5*critical/4+chi)
            # plt.ylim(0, 0.5)
            plt.xlabel('$Q$')
            plt.ylabel(r'$p(Q|k)$')
            plt.title(r'$\chi^2\ \mathrm{Distribution}$')
            plt.legend()
            plt.show()        
        